# Summary



# Note 

The **Save to database** bit have to be changed. See `all_by_all` for an example.

# Imports

In [1]:
__file__ = 'drug_pair_to_target_pair'

In [2]:
%load_ext autoreload
%autoreload 2

In [45]:
import common
common.configure_logging(level='debug')

DEBUG:root:Done configuring logging!


In [4]:
from biodata import *

/home/kimlab1/strokach/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [5]:
%matplotlib inline

In [6]:
%load_ext sql

/home/kimlab1/strokach/anaconda3/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/home/kimlab1/strokach/anaconda3/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [7]:
%sql mysql://strokach:@192.168.6.19/az_dream_2015

'Connected: strokach@az_dream_2015'

In [8]:
#SUFFIX = ''
SUFFIX = '_stitch'

# Functions

In [9]:
import string
import csv2sql

assert __file__

engine = sa.create_engine('mysql://strokach:@192.168.6.19:3306/az_dream_2015')

def attachment_path(name):
    return op.abspath(op.join(__file__, name))

def load_to_mysql(df, table_name, index_commands):
    tsv_file_name = op.abspath(op.join(__file__, table_name + '.tsv'))
    bz2_file_name = tsv_file_name + '.bz2'
    dtype = csv2sql.get_column_dtypes(df)
    df[:0].to_sql(table_name, engine, dtype=dtype, index=False, if_exists='replace')
    csv2sql.csv2sql.run_command('bzip2 -dkf "{}"'.format(bz2_file_name), '192.168.6.8')
    csv2sql.csv2sql.run_command(
        'mysql --local-infile -u strokach -h 192.168.6.19 az_dream_2015 -e '
        '"load data local infile \'{}\' into table {} ignore 1 lines"'.format(tsv_file_name, table_name))
    csv2sql.csv2sql.run_command('rm "{}"'.format(tsv_file_name), '192.168.6.8')
    for index_name, index_command in zip(string.ascii_letters, index_commands):
        columns, unique = index_command
        sql_command = (
            "create {unique} index {index_name} on {table_name} ({columns});"
            .format(
                unique='unique' if unique else '', 
                index_name=index_name,
                table_name=table_name,
                columns=", ".join(columns))
        )
        engine.execute(sql_command)

# Load data

In [10]:
# Read text file
drug_info_release = pd.read_csv(
    '../downloads/challenge_data/drug_synergy_data/drug_info_release.csv/'
    'Drug_info_release.csv', sep=','
)
drug_info_release_columns = list(drug_info_release.columns)

In [11]:
display(drug_info_release.head())
print(drug_info_release.shape)

,ChallengeName,Target(Official Symbol),HBA,cLogP,HBD,Lipinski,SMILES or PubChem ID,MW
0,ADAM17,ADAM17,NaN,NaN,NaN,NaN,NaN,NaN
1,AKT,"AKT1, AKT2, AKT3",8,1.18,5,0,c1cc(ccc1[C@H](CCO)NC(=O)C2(CCN(CC2)c3c4cc[nH]...,428.9
2,AKT_1,AKT*,6,3.24,3,0,c1ccc(cc1)c2cc3c(ccn4c3n[nH]c4=O)nc2c5ccc(cc5)...,407.5
3,AKT_PIK3C,"AKT*,PIK3C*",NaN,NaN,NaN,NaN,NaN,NaN
4,AKT_SGK,"AKT*,SGK*",NaN,NaN,NaN,NaN,NaN,NaN


(119, 8)


In [12]:
engine = sa.create_engine('mysql://strokach:@192.168.6.19:3306/az_dream')
drug_info_release_3 = pd.read_sql_table('drug_info_release_3', engine)

In [13]:
display(drug_info_release_3.head())
print(drug_info_release_3.shape)

,ChallengeName,Target(Official Symbol),HBA,cLogP,HBD,Lipinski,SMILES or PubChem ID,MW,AtomStereoCount,BondStereoCount,CID,CanonicalSMILES,Charge,Complexity,ConformerCount3D,ConformerModelRMSD3D,CovalentUnitCount,DefinedAtomStereoCount,DefinedBondStereoCount,EffectiveRotorCount3D,ExactMass,FeatureAcceptorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureCount3D,FeatureDonorCount3D,FeatureHydrophobeCount3D,FeatureRingCount3D,Fingerprint2D,HBondAcceptorCount,HBondDonorCount,HeavyAtomCount,IUPACName,InChI,InChIKey,IsomericSMILES,IsotopeAtomCount,MolecularFormula,MolecularWeight,MonoisotopicMass,RotatableBondCount,TPSA,UndefinedAtomStereoCount,UndefinedBondStereoCount,Volume3D,XLogP,XStericQuadrupole3D,YStericQuadrupole3D,ZStericQuadrupole3D,cids,targets,targets_bioentities,enzymes,enzymes_bioentities,transporters,transporters_bioentities,atcs
0,ADAM17,ADAM17,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None
1,AKT,"AKT1, AKT2, AKT3",8,1.18,5,0,c1cc(ccc1[C@H](CCO)NC(=O)C2(CCN(CC2)c3c4cc[nH]...,428.9,1,0,25227436,C1CN(CCC1(C(=O)NC(CCO)C2=CC=C(C=C2)Cl)N)C3=NC=...,0,580,256,1.0,1,1,0,8.2,428.172752,2,0,4,14,4,0,4,AAADceB7sAAEAAAAAAAAAAAAAAAAAWAAAAA8WIAAAAAAAF...,6,4,30,4-amino-N-[(1S)-1-(4-chlorophenyl)-3-hydroxypr...,InChI=1S/C21H25ClN6O2/c22-15-3-1-14(2-4-15)17(...,JDUBGYFRJFOXQC-KRWDZBQOSA-N,C1CN(CCC1(C(=O)N[C@@H](CCO)C2=CC=C(C=C2)Cl)N)C...,0,C21H25ClN6O2,428.91520,428.172752,6,120.0,0,0,319.3,1.7,14.27,5.17,1.08,"25227436,42602260,57750340",None,None,None,None,None,None,None
2,AKT_1,AKT*,6,3.24,3,0,c1ccc(cc1)c2cc3c(ccn4c3n[nH]c4=O)nc2c5ccc(cc5)...,407.5,0,0,24964624,C1CC(C1)(C2=CC=C(C=C2)C3=C(C=C4C(=N3)C=CN5C4=N...,0,760,4,0.8,1,0,0,3.8,407.174610,2,0,1,11,2,0,6,AAADceB7oAAAAAAAAAAAAAAAAABgAQAAAAA8YIEAAAAAAE...,4,2,31,8-[4-(1-aminocyclobutyl)phenyl]-9-phenyl-2H-[1...,InChI=1S/C25H21N5O/c26-25(12-4-13-25)18-9-7-17...,ULDXWLCXEDXJGE-UHFFFAOYSA-N,C1CC(C1)(C2=CC=C(C=C2)C3=C(C=C4C(=N3)C=CN5C4=N...,0,C25H21N5O,407.46714,407.174610,3,83.6,0,0,317.2,3.0,16.28,4.11,1.07,24964624,None,None,None,None,None,None,None
3,AKT_PIK3C,"AKT*,PIK3C*",NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None
4,AKT_SGK,"AKT*,SGK*",NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None


(119, 57)


## Load drug info data

In [14]:
engine = sa.create_engine('mysql://strokach:@192.168.6.19:3306/az_dream')
drug_to_cid = pd.read_sql('drug_to_cid', engine)

engine = sa.create_engine('mysql://strokach:@192.168.6.19:3306/az_dream_2015')
all_drug_pairs = pd.read_sql('DRUG_PAIRS', engine)
drug_to_target = pd.read_sql('drug_to_target' + SUFFIX, engine)

In [15]:
all_drug_pairs.head(2)

,d_1,d_2
0,ADAM17,ADAM17
1,ADAM17,AKT


In [16]:
all_drug_pairs['unique_id'] = all_drug_pairs['d_1'] + '.' + all_drug_pairs['d_2']
all_drug_pairs.sort_values(['d_1', 'd_2'], inplace=True)

In [17]:
all_drug_pairs.head(2)

,d_1,d_2,unique_id
0,ADAM17,ADAM17,ADAM17.ADAM17
1,ADAM17,AKT,ADAM17.AKT


In [18]:
drug_to_cid.head(2)

,ChallengeName,cid
0,AKT,25227436
1,AKT,57750340


In [19]:
drug_to_cid.dtypes

ChallengeName    object
cid               int64
dtype: object

In [20]:
drug_to_target.head(2)

,drug,target_original,target,hgnc_name,hgnc_acc,hgnc_description,ensg,enst,ensp,ensp_idx
0,AKT,"AKT1, AKT2, AKT3","AKT1, AKT2, AKT3",RTN1,HGNC:10467,reticulon 1,ENSG00000139970,ENST00000267484,ENSP00000267484,267484
1,AKT,"AKT1, AKT2, AKT3","AKT1, AKT2, AKT3",RTN4,HGNC:14085,reticulon 4,ENSG00000115310,ENST00000337526,ENSP00000337838,337838


# Connect to database

In [49]:
from importlib import reload
import csv2sql
reload(csv2sql)
import logging
db = csv2sql.DataFrameToMySQL(
    'mysql://strokach:@192.168.6.19:3306/az_dream_2015', 
    'drug_pair_to_target_pair', 
    '192.168.233.20', 
    echo=False
)

DEBUG:sqlalchemy.pool.QueuePool:Created new connection <_mysql.connection open to '192.168.6.19' at 2e3f978>
DEBUG:sqlalchemy.pool.QueuePool:Connection <_mysql.connection open to '192.168.6.19' at 2e3f978> checked out from pool
DEBUG:sqlalchemy.pool.QueuePool:Connection <_mysql.connection open to '192.168.6.19' at 2e3f978> being returned to pool
DEBUG:sqlalchemy.pool.QueuePool:Connection <_mysql.connection open to '192.168.6.19' at 2e3f978> rollback-on-return


# Tables

## drug_pair_to_cid_pair

In [22]:
drug_pair_to_cid_pair = (
    all_drug_pairs
    .merge(
        drug_to_cid[['ChallengeName', 'cid']]
            .rename(columns={'ChallengeName': 'd_1', 'cid': 'cid_1'}), 
        on=['d_1'])
    .merge(
        drug_to_cid[['ChallengeName', 'cid']]
            .rename(columns={'ChallengeName': 'd_2', 'cid': 'cid_2'}), 
        on=['d_2'])
)
#
idx = (drug_pair_to_cid_pair['cid_1'] > drug_pair_to_cid_pair['cid_2'])
(drug_pair_to_cid_pair.loc[idx, 'cid_1'], drug_pair_to_cid_pair.loc[idx, 'cid_2']) = (
    drug_pair_to_cid_pair.loc[idx, 'cid_2'], drug_pair_to_cid_pair.loc[idx, 'cid_1']
)
drug_pair_to_cid_pair['cid_pair'] = (
    drug_pair_to_cid_pair['cid_1'].astype(int).astype(str) + ',' + 
    drug_pair_to_cid_pair['cid_2'].astype(int).astype(str)
)

In [23]:
print('{:,d}'.format(drug_pair_to_cid_pair.shape[0]))
drug_pair_to_cid_pair = drug_pair_to_cid_pair.drop_duplicates()
print('{:,d}'.format(drug_pair_to_cid_pair.drop_duplicates().shape[0]))

832,613
789,395


In [24]:
drug_pair_to_cid_pair.head(2)

,d_1,d_2,unique_id,cid_1,cid_2,cid_pair
0,AKT,AKT,AKT.AKT,25227436,25227436,"25227436,25227436"
1,AKT,AKT,AKT.AKT,25227436,57750340,"25227436,57750340"


In [25]:
drug_pair_to_cid_pair.dtypes

d_1          object
d_2          object
unique_id    object
cid_1         int64
cid_2         int64
cid_pair     object
dtype: object

In [26]:
assert not any(drug_pair_to_cid_pair['cid_1'] > drug_pair_to_cid_pair['cid_2'])

In [27]:
# Save to database
db.import_table(
    drug_pair_to_cid_pair, 
    'drug_pair_to_cid_pair' + SUFFIX, 
    [(('unique_id', 'd_1', 'd_2'), False), 
     (('d_1', 'd_2', 'unique_id'), False), 
     (('unique_id', 'cid_1', 'cid_2'), False), 
     (('cid_1', 'cid_2', 'unique_id'), False), 
     (('unique_id', 'cid_pair'), False), 
     (('cid_pair', 'unique_id'), False)] 
)

DEBUG:csv2sql.core:Uncompressing file...
INFO:csv2sql.core:bzip2 -dkf '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/drug_pair_to_target_pair/drug_pair_to_cid_pair_stitch.tsv.bz2'
DEBUG:csv2sql.core:Running on host: '192.168.233.20'
DEBUG:csv2sql.core:Initializing SSH client: '192.168.233.20'
DEBUG:csv2sql.core:Command ran successfully!
DEBUG:csv2sql.core:output: 


FML
FML
FML

DEBUG:csv2sql.core:Loading data into MySQL table: 'drug_pair_to_cid_pair_stitch'...
INFO:csv2sql.core:mysql --local-infile -h 192.168.6.19 -P 3306 -u strokach  az_dream_2015 -e "load data local infile '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/drug_pair_to_target_pair/drug_pair_to_cid_pair_stitch.tsv' into table `drug_pair_to_cid_pair_stitch` ignore 1 lines;  show warnings;" 
DEBUG:csv2sql.core:Running locally
DEBUG:csv2sql.core:Command ran successfully!
DEBUG:csv2sql.core:output: 
DEBUG:csv2sql.core:Removing uncompressed file '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/drug_pair_to_target_pair/drug_pair_to_cid_pair_stitch.tsv'...
INFO:csv2sql.core:rm -f '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/drug_pair_to_target_pair/drug_pair_to_cid_pair_stitch.tsv'
DEBUG:csv2sql.core:Running locally
DEBUG:csv2sql.core:Command ran successfully!
DEBUG:csv2sql.core:output: 


## drug_pair_to_ensp_pair

In [28]:
drug_pair_to_ensp_pair = (
    all_drug_pairs
    .merge(
        drug_to_target[['drug', 'ensp']]
            .rename(columns={'drug': 'd_1', 'ensp': 'ensp_1'}), 
        on=['d_1'])
    .merge(
        drug_to_target[['drug', 'ensp']]
            .rename(columns={'drug': 'd_2', 'ensp': 'ensp_2'}), 
        on=['d_2'])
)
#
idx = (drug_pair_to_ensp_pair['ensp_1'] > drug_pair_to_ensp_pair['ensp_2'])
(drug_pair_to_ensp_pair.loc[idx, 'ensp_1'], drug_pair_to_ensp_pair.loc[idx, 'ensp_2']) = (
    drug_pair_to_ensp_pair.loc[idx, 'ensp_2'], drug_pair_to_ensp_pair.loc[idx, 'ensp_1']
)
drug_pair_to_ensp_pair['ensp_pair'] = (
    drug_pair_to_ensp_pair['ensp_1'] + ',' + drug_pair_to_ensp_pair['ensp_2']
)

In [29]:
print('{:,d}'.format(drug_pair_to_ensp_pair.shape[0]))
drug_pair_to_ensp_pair = drug_pair_to_ensp_pair.drop_duplicates()
print('{:,d}'.format(drug_pair_to_ensp_pair.drop_duplicates().shape[0]))

50,668,566
47,699,947


In [30]:
display(drug_pair_to_ensp_pair.head())

,d_1,d_2,unique_id,ensp_1,ensp_2,ensp_pair
0,ADAM17,ADAM17,ADAM17.ADAM17,ENSP00000309968,ENSP00000309968,"ENSP00000309968,ENSP00000309968"
1,ADAM17,ADAM17,ADAM17.ADAM17,ENSP00000309968,ENSP00000418728,"ENSP00000309968,ENSP00000418728"
2,ADAM17,ADAM17,ADAM17.ADAM17,ENSP00000309968,ENSP00000480552,"ENSP00000309968,ENSP00000480552"
4,ADAM17,ADAM17,ADAM17.ADAM17,ENSP00000418728,ENSP00000418728,"ENSP00000418728,ENSP00000418728"
5,ADAM17,ADAM17,ADAM17.ADAM17,ENSP00000418728,ENSP00000480552,"ENSP00000418728,ENSP00000480552"


In [31]:
# Save to database
db.import_table(
    drug_pair_to_ensp_pair, 
    'drug_pair_to_ensp_pair' + SUFFIX, 
    [(('unique_id', 'd_1', 'd_2'), False), 
     (('d_1', 'd_2', 'unique_id'), False), 
     (('unique_id', 'ensp_1', 'ensp_2'), False), 
     (('ensp_1', 'ensp_2', 'unique_id'), False), 
     (('unique_id', 'ensp_pair'), False), 
     (('ensp_pair', 'unique_id'), False)] 
)

DEBUG:csv2sql.core:Uncompressing file...
INFO:csv2sql.core:bzip2 -dkf '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/drug_pair_to_target_pair/drug_pair_to_ensp_pair_stitch.tsv.bz2'
DEBUG:csv2sql.core:Running on host: '192.168.233.20'
DEBUG:csv2sql.core:Command ran successfully!
DEBUG:csv2sql.core:output: 
DEBUG:csv2sql.core:Loading data into MySQL table: 'drug_pair_to_ensp_pair_stitch'...
INFO:csv2sql.core:mysql --local-infile -h 192.168.6.19 -P 3306 -u strokach  az_dream_2015 -e "load data local infile '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/drug_pair_to_target_pair/drug_pair_to_ensp_pair_stitch.tsv' into table `drug_pair_to_ensp_pair_stitch` ignore 1 lines;  show warnings;" 
DEBUG:csv2sql.core:Running locally
DEBUG:csv2sql.core:Command ran successfully!
DEBUG:csv2sql.core:output: 
DEBUG:csv2sql.core:Removing uncompressed file '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/drug_pair_to_target_pair/drug_pair_to_en

## drug_pair_to_ensp_idx_pair

In [32]:
drug_pair_to_ensp_idx_pair = (
    all_drug_pairs
    .merge(
        drug_to_target[['drug', 'ensp_idx']]
            .rename(columns={'drug': 'd_1', 'ensp_idx': 'ensp_idx_1'}), 
        on=['d_1'])
    .merge(
        drug_to_target[['drug', 'ensp_idx']]
            .rename(columns={'drug': 'd_2', 'ensp_idx': 'ensp_idx_2'}), 
        on=['d_2'])
)

In [33]:
idx = (drug_pair_to_ensp_idx_pair['ensp_idx_1'] > drug_pair_to_ensp_idx_pair['ensp_idx_2'])
(drug_pair_to_ensp_idx_pair.loc[idx, 'ensp_idx_1'], drug_pair_to_ensp_idx_pair.loc[idx, 'ensp_idx_2']) = (
    drug_pair_to_ensp_idx_pair.loc[idx, 'ensp_idx_2'], drug_pair_to_ensp_idx_pair.loc[idx, 'ensp_idx_1']
)
drug_pair_to_ensp_idx_pair['ensp_idx_pair'] = (
    drug_pair_to_ensp_idx_pair['ensp_idx_1'].astype(int).astype(str) + ',' + 
    drug_pair_to_ensp_idx_pair['ensp_idx_2'].astype(int).astype(str)
)

In [34]:
print('{:,d}'.format(drug_pair_to_ensp_idx_pair.shape[0]))
drug_pair_to_ensp_idx_pair = drug_pair_to_ensp_idx_pair.drop_duplicates()
print('{:,d}'.format(drug_pair_to_ensp_idx_pair.shape[0]))

50,668,566
47,699,947


In [35]:
display(drug_pair_to_ensp_idx_pair.head())

,d_1,d_2,unique_id,ensp_idx_1,ensp_idx_2,ensp_idx_pair
0,ADAM17,ADAM17,ADAM17.ADAM17,309968,309968,"309968,309968"
1,ADAM17,ADAM17,ADAM17.ADAM17,309968,418728,"309968,418728"
2,ADAM17,ADAM17,ADAM17.ADAM17,309968,480552,"309968,480552"
4,ADAM17,ADAM17,ADAM17.ADAM17,418728,418728,"418728,418728"
5,ADAM17,ADAM17,ADAM17.ADAM17,418728,480552,"418728,480552"


In [36]:
# Save to database
db.import_table(
    drug_pair_to_ensp_idx_pair, 
    'drug_pair_to_ensp_idx_pair' + SUFFIX, 
    [(('unique_id', 'd_1', 'd_2'), False), 
     (('d_1', 'd_2', 'unique_id'), False), 
     (('unique_id', 'ensp_idx_1', 'ensp_idx_2'), False), 
     (('ensp_idx_1', 'ensp_idx_2', 'unique_id'), False), 
     (('unique_id', 'ensp_idx_pair'), False), 
     (('ensp_idx_pair', 'UNIQUE_ID'), False)] 
)

DEBUG:csv2sql.core:Uncompressing file...
INFO:csv2sql.core:bzip2 -dkf '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/drug_pair_to_target_pair/drug_pair_to_ensp_idx_pair_stitch.tsv.bz2'
DEBUG:csv2sql.core:Running on host: '192.168.233.20'
DEBUG:csv2sql.core:Command ran successfully!
DEBUG:csv2sql.core:output: 
DEBUG:csv2sql.core:Loading data into MySQL table: 'drug_pair_to_ensp_idx_pair_stitch'...
INFO:csv2sql.core:mysql --local-infile -h 192.168.6.19 -P 3306 -u strokach  az_dream_2015 -e "load data local infile '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/drug_pair_to_target_pair/drug_pair_to_ensp_idx_pair_stitch.tsv' into table `drug_pair_to_ensp_idx_pair_stitch` ignore 1 lines;  show warnings;" 
DEBUG:csv2sql.core:Running locally
DEBUG:csv2sql.core:Command ran successfully!
DEBUG:csv2sql.core:output: 
DEBUG:csv2sql.core:Removing uncompressed file '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/drug_pair_to_target_pair

## drug_pair_to_gene_pair

In [37]:
drug_pair_to_gene_pair = (
    all_drug_pairs
    .merge(
        drug_to_target[['drug', 'hgnc_name']]
            .rename(columns={'drug': 'd_1', 'hgnc_name': 'gene_1'}), 
        on=['d_1'])
    .merge(
        drug_to_target[['drug', 'hgnc_name']]
            .rename(columns={'drug': 'd_2', 'hgnc_name': 'gene_2'}), 
        on=['d_2'])
)
#
idx = (drug_pair_to_gene_pair['gene_1'] > drug_pair_to_gene_pair['gene_2'])
(drug_pair_to_gene_pair.loc[idx, 'gene_1'], drug_pair_to_gene_pair.loc[idx, 'gene_2']) = (
    drug_pair_to_gene_pair.loc[idx, 'gene_2'], drug_pair_to_gene_pair.loc[idx, 'gene_1']
)
drug_pair_to_gene_pair['gene_pair'] = (
    drug_pair_to_gene_pair['gene_1'] + ',' + drug_pair_to_gene_pair['gene_2']
)

In [38]:
print('{:,d}'.format(drug_pair_to_gene_pair.shape[0]))
drug_pair_to_gene_pair = drug_pair_to_gene_pair.drop_duplicates()
print('{:,d}'.format(drug_pair_to_gene_pair.shape[0]))

50,668,566
38,489,245


In [39]:
display(drug_pair_to_gene_pair.head())

,d_1,d_2,unique_id,gene_1,gene_2,gene_pair
0,ADAM17,ADAM17,ADAM17.ADAM17,ADAM17,ADAM17,"ADAM17,ADAM17"
9,ADAM17,AKT,ADAM17.AKT,ADAM17,RTN1,"ADAM17,RTN1"
10,ADAM17,AKT,ADAM17.AKT,ADAM17,RTN4,"ADAM17,RTN4"
11,ADAM17,AKT,ADAM17.AKT,ADAM17,BEX1,"ADAM17,BEX1"
18,AKT,AKT,AKT.AKT,RTN1,RTN1,"RTN1,RTN1"


In [67]:
drug_pair_to_gene_pair.head()

,d_1,d_2,unique_id,gene_1,gene_2,gene_pair
0,ADAM17,ADAM17,ADAM17.ADAM17,ADAM17,ADAM17,"ADAM17,ADAM17"
9,ADAM17,AKT,ADAM17.AKT,ADAM17,RTN1,"ADAM17,RTN1"
10,ADAM17,AKT,ADAM17.AKT,ADAM17,RTN4,"ADAM17,RTN4"
11,ADAM17,AKT,ADAM17.AKT,ADAM17,BEX1,"ADAM17,BEX1"
18,AKT,AKT,AKT.AKT,RTN1,RTN1,"RTN1,RTN1"


In [68]:
# Save to database
db.import_table(
    drug_pair_to_gene_pair, 
    'drug_pair_to_gene_pair_blahblah' + SUFFIX, 
    [(('unique_id', 'd_1', 'd_2'), False), 
     (('d_1', 'd_2', 'unique_id'), False), 
     (('unique_id', 'gene_1', 'gene_2'), False), 
     (('gene_1', 'gene_2', 'unique_id'), False), 
     (('unique_id', 'gene_pair'), False), 
     (('gene_pair', 'unique_id'), False)]
)

DEBUG:sqlalchemy.pool.QueuePool:Connection <_mysql.connection open to '192.168.6.19' at 2e3f978> checked out from pool
DEBUG:sqlalchemy.pool.QueuePool:Connection <_mysql.connection open to '192.168.6.19' at 2e3f978> being returned to pool
DEBUG:sqlalchemy.pool.QueuePool:Connection <_mysql.connection open to '192.168.6.19' at 2e3f978> rollback-on-return
DEBUG:sqlalchemy.pool.QueuePool:Connection <_mysql.connection open to '192.168.6.19' at 2e3f978> checked out from pool
DEBUG:sqlalchemy.pool.QueuePool:Connection <_mysql.connection open to '192.168.6.19' at 2e3f978> being returned to pool
DEBUG:sqlalchemy.pool.QueuePool:Connection <_mysql.connection open to '192.168.6.19' at 2e3f978> rollback-on-return
DEBUG:sqlalchemy.pool.QueuePool:Connection <_mysql.connection open to '192.168.6.19' at 2e3f978> checked out from pool
DEBUG:sqlalchemy.pool.QueuePool:Connection <_mysql.connection open to '192.168.6.19' at 2e3f978> being returned to pool
DEBUG:sqlalchemy.pool.QueuePool:Connection <_mysql.

In [66]:
step = 7
batch_size = 128
train_labels = 1000
(step * batch_size) % (train_labels- batch_size)

24

In [43]:
print('\nCREATE TABLE drug_pair_to_gene_pair_stitch (\n\td_1 VARCHAR(255), \n\td_2 VARCHAR(255), \n\tunique_id VARCHAR(255), \n\tgene_1 VARCHAR(255), \n\tgene_2 VARCHAR(255), \n\tgene_pair VARCHAR(255)\n)\n\n')


CREATE TABLE drug_pair_to_gene_pair_stitch (
	d_1 VARCHAR(255), 
	d_2 VARCHAR(255), 
	unique_id VARCHAR(255), 
	gene_1 VARCHAR(255), 
	gene_2 VARCHAR(255), 
	gene_pair VARCHAR(255)
)




# Finalize

In [ ]:
import datetime 
print(datetime.datetime.now())